In [2]:
# ====================
# OASIS CN vs MCI
# ====================
import os, random, warnings
import numpy as np
from tqdm import tqdm
warnings.filterwarnings('ignore')

# Torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torch.nn.functional as F

# Vision
import torchvision.transforms as T
import torchvision.models as models

# Imaging
import nibabel as nib
from PIL import Image
import cv2
from skimage import exposure

# Metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score, roc_curve, auc
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

# Colab Drive
from google.colab import drive

# ==================== CONFIGURATION ====================
class Config:
    DATA_ROOT = "/content/drive/MyDrive/OASIS3_Preprocessed_Data"
    IMAGE_SIZE = 256
    BATCH_SIZE = 32
    NUM_EPOCHS = 60
    LEARNING_RATE = 3e-5
    WEIGHT_DECAY = 1e-4

    # 2.5D multi-slice extraction
    USE_25D = True
    SLAB_OFFSETS = [-12, -6, 0, 6, 12]  # anchors
    SLAB_SAMPLES = 3  # random 3 each epoch

    # Class balancing - calculated dynamically
    USE_DYNAMIC_WEIGHTS = True
    BASE_MCI_WEIGHT = 1.8  # Base multiplier for MCI class

    # IMPROVED Loss function
    USE_FOCAL_LOSS = True
    FOCAL_ALPHA = 0.85  # Slightly favor minority class
    FOCAL_GAMMA = 1.5   # Reduced gamma for better convergence
    LABEL_SMOOTHING = 0.05  # Reduced for better training

    # Augmentation
    USE_MIXUP = True
    MIXUP_ALPHA = 0.2
    USE_CUTMIX = True
    CUTMIX_PROB = 0.2

    # Threshold optimization
    OPTIMIZE_THRESHOLD = True

    # Model
    MODEL_TYPE = 'convnext'

    # IMPROVED Training
    PATIENCE = 15
    WARMUP_EPOCHS = 5  # Gradual learning rate warmup
    MIN_DELTA = 0.001  # Minimum improvement for early stopping

    CLASSES = ['CN', 'MCI']
    NUM_CLASSES = 2

cfg = Config()

# ==================== SETUP ====================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("="*80)
print("OASIS CN vs MCI Pipeline")
print("="*80)
print(f"✓ Device: {device}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

drive.mount("/content/drive")

# ==================== PREPROCESSING ====================
class VolumePreprocessor:
    @staticmethod
    def extract_25d_slabs(volume):
        if volume.ndim == 4:
            volume = volume[..., 0]
        if volume.ndim != 3:
            return None
        h, w, d = volume.shape
        center = d // 2
        offsets = random.sample(cfg.SLAB_OFFSETS, cfg.SLAB_SAMPLES)
        slabs = []
        for off in offsets:
            idx = int(np.clip(center + off, 0, d-1))
            slice_img = volume[:, :, idx]
            # robust per-slice normalization
            p1, p99 = np.percentile(slice_img, [1, 99])
            slice_img = np.clip(slice_img, p1, p99)
            slice_img = (slice_img - p1) / (p99 - p1 + 1e-8)
            slice_img = exposure.equalize_adapthist(slice_img, clip_limit=0.03)
            slabs.append(slice_img)
        rgb = np.stack(slabs, axis=-1)
        rgb = (rgb * 255).astype(np.uint8)
        return rgb

# ==================== DATASET ====================
class OASISDataset(Dataset):
    def __init__(self, files, labels, mode='train'):
        self.files = files
        self.labels = labels
        self.mode = mode
        self.preprocessor = VolumePreprocessor()

        if mode == 'train':
            self.transform = T.Compose([
                T.Resize((cfg.IMAGE_SIZE+32, cfg.IMAGE_SIZE+32)),
                T.RandomCrop(cfg.IMAGE_SIZE),
                T.RandomHorizontalFlip(p=0.5),
                T.RandomRotation(10),
                T.ColorJitter(brightness=0.1, contrast=0.1),
                T.ToTensor(),
                T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ])
        else:
            self.transform = T.Compose([
                T.Resize((cfg.IMAGE_SIZE, cfg.IMAGE_SIZE)),
                T.ToTensor(),
                T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        path = self.files[idx]
        label = int(self.labels[idx])

        try:
            volume = nib.load(path).get_fdata()
            if cfg.USE_25D:
                rgb = self.preprocessor.extract_25d_slabs(volume)
            else:
                slice_img = volume[:,:,volume.shape[2]//2]
                p1, p99 = np.percentile(slice_img, [1, 99])
                slice_img = np.clip(slice_img, p1, p99)
                slice_img = (slice_img - p1) / (p99 - p1 + 1e-8)
                slice_img = exposure.equalize_adapthist(slice_img, clip_limit=0.03)
                rgb = np.stack([slice_img]*3,axis=-1)
                rgb = (rgb*255).astype(np.uint8)
        except Exception as e:
            print(f"Warning: Error loading {path}: {e}")
            # rare I/O hiccup -> return a small noise image to keep batch rolling
            rgb = np.random.randint(0,255,(cfg.IMAGE_SIZE,cfg.IMAGE_SIZE,3),dtype=np.uint8)

        pil_img = Image.fromarray(rgb)
        img_tensor = self.transform(pil_img)
        return img_tensor, label

# ==================== LOSS FUNCTIONS ====================
class ImprovedFocalLoss(nn.Module):
    """Focal Loss with proper tensor dimensions"""
    def __init__(self, alpha=0.75, gamma=2.0, smoothing=0.05):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.smoothing = smoothing

    def forward(self, inputs, targets):
        # Apply label smoothing
        n_classes = inputs.size(-1)
        onehot = F.one_hot(targets, n_classes).to(dtype=inputs.dtype)
        onehot = onehot * (1 - self.smoothing) + self.smoothing / n_classes

        # Compute cross entropy loss
        log_pt = F.log_softmax(inputs, dim=1)
        ce_loss = -(onehot * log_pt).sum(dim=1)

        # Compute pt for focal weight
        pt = torch.exp(-ce_loss)

        # Focal weight
        focal_weight = (1 - pt) ** self.gamma  # Shape: (batch_size,)

        # Alpha weighting (favor minority class)
        alpha_weight = torch.where(targets == 1, self.alpha, 1 - self.alpha)  # Shape: (batch_size,)

        # Combine losses - all tensors now have shape (batch_size,)
        loss = alpha_weight * focal_weight * ce_loss

        return loss.mean()

class CombinedLoss(nn.Module):
    """Combine Focal Loss with standard CrossEntropy for stability"""
    def __init__(self, focal_weight=0.7, ce_weight=0.3):
        super().__init__()
        self.focal_loss = ImprovedFocalLoss(alpha=cfg.FOCAL_ALPHA, gamma=cfg.FOCAL_GAMMA,
                                          smoothing=cfg.LABEL_SMOOTHING)
        self.ce_loss = nn.CrossEntropyLoss(label_smoothing=cfg.LABEL_SMOOTHING)
        self.focal_weight = focal_weight
        self.ce_weight = ce_weight

    def forward(self, inputs, targets):
        focal = self.focal_loss(inputs, targets)
        ce = self.ce_loss(inputs, targets)
        return self.focal_weight * focal + self.ce_weight * ce

# ==================== MODEL ====================
def create_model(model_type='convnext'):
    if model_type == 'convnext':
        try:
            weights = models.ConvNeXt_Tiny_Weights.IMAGENET1K_V1
            model = models.convnext_tiny(weights=weights)
        except Exception:
            model = models.convnext_tiny(pretrained=True)
        num_features = model.classifier[2].in_features
        # Classifier with better regularization
        model.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(1),
            nn.LayerNorm(num_features),
            nn.Dropout(0.4),
            nn.Linear(num_features, 512),
            nn.GELU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.GELU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.2),
            nn.Linear(256, cfg.NUM_CLASSES)
        )
    elif model_type == 'efficientnet_v2':
        try:
            weights = models.EfficientNet_V2_S_Weights.IMAGENET1K_V1
            model = models.efficientnet_v2_s(weights=weights)
        except Exception:
            model = models.efficientnet_v2_s(pretrained=True)
        num_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(num_features, 512), nn.SiLU(),
            nn.BatchNorm1d(512), nn.Dropout(0.3),
            nn.Linear(512, 256), nn.SiLU(),
            nn.BatchNorm1d(256), nn.Dropout(0.2),
            nn.Linear(256, cfg.NUM_CLASSES)
        )
    return model.to(device)

# ==================== TRAINING UTILS ====================
def mixup_data(x,y,alpha=0.2):
    if alpha<=0: return x,y,y,1.0
    lam = np.random.beta(alpha,alpha)
    index = torch.randperm(x.size(0),device=x.device)
    mixed_x = lam*x+(1-lam)*x[index]
    return mixed_x, y, y[index], lam

def cutmix_data(x,y,prob=0.2):
    if random.random()>=prob: return x,y,y,1.0
    lam = np.random.beta(1.0,1.0)
    B,C,W,H=x.size()
    index = torch.randperm(B,device=x.device)
    cut_rat=np.sqrt(1.-lam)
    cut_w=int(W*cut_rat); cut_h=int(H*cut_rat)
    cx=np.random.randint(W); cy=np.random.randint(H)
    bbx1=np.clip(cx-cut_w//2,0,W); bby1=np.clip(cy-cut_h//2,0,H)
    bbx2=np.clip(cx+cut_w//2,0,W); bby2=np.clip(cy+cut_h//2,0,H)
    x[:,:,bbx1:bbx2,bby1:bby2]=x[index,:,bbx1:bbx2,bby1:bby2]
    lam=1-((bbx2-bbx1)*(bby2-bby1)/(W*H))
    return x,y,y[index],lam

# ==================== TRAIN / EVAL ====================
def train_epoch(model, loader, criterion, optimizer, epoch):
    model.train()
    total_loss, all_preds, all_labels = 0, [], []

    # Progress bar
    pbar = tqdm(loader, desc=f"Epoch {epoch+1}", leave=False)

    for batch_idx, (inputs, labels) in enumerate(pbar):
        inputs, labels = inputs.to(device), labels.to(device)

        # Apply augmentations with lower probability in early epochs
        aug_prob = min(0.7, 0.3 + (epoch / 10) * 0.4)
        r = random.random()

        if cfg.USE_MIXUP and r < aug_prob * 0.5:
            inputs, y_a, y_b, lam = mixup_data(inputs, labels, alpha=cfg.MIXUP_ALPHA)
            outputs = model(inputs)
            loss = lam * criterion(outputs, y_a) + (1 - lam) * criterion(outputs, y_b)
        elif cfg.USE_CUTMIX and r < aug_prob * 0.7:
            inputs, y_a, y_b, lam = cutmix_data(inputs, labels, prob=cfg.CUTMIX_PROB)
            outputs = model(inputs)
            loss = lam * criterion(outputs, y_a) + (1 - lam) * criterion(outputs, y_b)
        else:
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()

        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        total_loss += loss.item()
        all_preds.extend(outputs.argmax(1).detach().cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # Update progress bar
        if batch_idx % 10 == 0:
            pbar.set_postfix({
                'Loss': f'{loss.item():.3f}',
                'Acc': f'{accuracy_score(all_labels, all_preds):.3f}'
            })

    train_acc = accuracy_score(all_labels, all_preds)
    train_bal = balanced_accuracy_score(all_labels, all_preds)

    return total_loss / len(loader), train_acc, train_bal

def evaluate(model, loader, criterion, return_probs=False):
    model.eval()
    all_probs, all_labels = [], []
    total_loss = 0

    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Evaluating", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            probs = F.softmax(outputs, dim=1)
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_probs = np.array(all_probs)
    all_labels = np.array(all_labels)
    mci_probs = all_probs[:, 1]

    # Optimize threshold for balanced accuracy
    if cfg.OPTIMIZE_THRESHOLD and len(np.unique(all_labels)) > 1:
        fpr, tpr, thresholds = roc_curve(all_labels, mci_probs)
        if len(thresholds) > 0:
            # Find threshold that maximizes balanced accuracy
            balanced_acc_scores = []
            for thr in thresholds:
                preds = (mci_probs >= thr).astype(int)
                bal_acc = balanced_accuracy_score(all_labels, preds)
                balanced_acc_scores.append(bal_acc)

            best_idx = np.argmax(balanced_acc_scores)
            thr_opt = thresholds[best_idx]
            preds = (mci_probs >= thr_opt).astype(int)
        else:
            thr_opt = 0.5
            preds = all_probs.argmax(1)
    else:
        thr_opt = 0.5
        preds = all_probs.argmax(1)

    # Calculate comprehensive metrics
    acc = accuracy_score(all_labels, preds)
    bal_acc = balanced_accuracy_score(all_labels, preds)
    precision = precision_score(all_labels, preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, preds, average='weighted', zero_division=0)
    cm = confusion_matrix(all_labels, preds)

    results = {
        'loss': total_loss / len(loader),
        'accuracy': acc,
        'balanced_accuracy': bal_acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': cm,
        'threshold': thr_opt
    }

    if return_probs:
        results['probabilities'] = mci_probs
        results['labels'] = all_labels

    return results

# ==================== CHECKPOINTS ====================
def save_checkpoint(model, optimizer, scheduler, epoch, metrics, path="best.pth"):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'epoch': epoch,
        'metrics': metrics
    }
    torch.save(checkpoint, path)

def load_checkpoint(path="best.pth", map_location=device):
    if not os.path.exists(path):
        return None
    try:
        return torch.load(path, map_location=map_location, weights_only=False)
    except Exception as e:
        print(f"Warning: Could not load checkpoint '{path}': {e}")
        return None

# ==================== MAIN ====================
def main():
    # Collect files
    files, labels = [], []
    for i, cls in enumerate(cfg.CLASSES):
        d = os.path.join(cfg.DATA_ROOT, cls)
        if not os.path.isdir(d):
            continue
        for f in os.listdir(d):
            if f.endswith(('.nii', '.nii.gz')):
                files.append(os.path.join(d, f))
                labels.append(i)

    files = np.array(files)
    labels = np.array(labels)

    print(f"✓ Initially found {len(files)} files | CN={sum(labels==0)}, MCI={sum(labels==1)}")

    # RANDOM SAMPLING OF CN CASES
    MAX_CN_SAMPLES = 440  # Cap number of CN samples to overcome huge class imbalance

    if sum(labels==0) > MAX_CN_SAMPLES:
        cn_indices = np.where(labels == 0)[0]
        mci_indices = np.where(labels == 1)[0]

        print(f"📊 Original CN samples: {len(cn_indices)}")
        print(f"🎯 Randomly sampling {MAX_CN_SAMPLES} CN cases...")

        # Randomly sample CN cases
        np.random.seed(42)  # For reproducibility
        sampled_cn_indices = np.random.choice(cn_indices, size=MAX_CN_SAMPLES, replace=False)

        # Combine sampled CN with all MCI
        selected_indices = np.concatenate([sampled_cn_indices, mci_indices])

        # Update files and labels
        files = files[selected_indices]
        labels = labels[selected_indices]

        print(f"✅ Balanced dataset: CN={sum(labels==0)}, MCI={sum(labels==1)}")
    else:
        print(f"✓ Using all available samples: CN={sum(labels==0)}, MCI={sum(labels==1)}")

    # Check final class distribution
    cn_count = sum(labels == 0)
    mci_count = sum(labels == 1)
    imbalance_ratio = cn_count / mci_count if mci_count > 0 else float('inf')
    print(f"📊 Final class imbalance ratio: {imbalance_ratio:.2f}:1 (CN:MCI)")

    # Splits
    X_temp, X_test, y_temp, y_test = train_test_split(
        files, labels, test_size=0.15, stratify=labels, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.1765, stratify=y_temp, random_state=42
    )

    # Datasets
    train_ds = OASISDataset(X_train, y_train, 'train')
    val_ds = OASISDataset(X_val, y_val, 'val')
    test_ds = OASISDataset(X_test, y_test, 'test')

    # Calculate class weights properly
    class_counts = np.bincount(y_train)
    print(f"✓ Training class distribution: CN={class_counts[0]}, MCI={class_counts[1]}")

    # Improved dynamic weighting
    if cfg.USE_DYNAMIC_WEIGHTS:
        # Calculate inverse frequency weights
        total_samples = len(y_train)
        weights = total_samples / (len(class_counts) * class_counts)
        # Apply additional boost to minority class
        weights[1] *= cfg.BASE_MCI_WEIGHT
        print(f"✓ Class weights: CN={weights[0]:.3f}, MCI={weights[1]:.3f}")
    else:
        weights = np.array([1.0, cfg.BASE_MCI_WEIGHT])

    # Create sample weights for balanced sampling
    sample_weights = weights[y_train]
    sampler = WeightedRandomSampler(
        torch.from_numpy(sample_weights).double(),
        len(sample_weights),
        replacement=True
    )

    # Data loaders
    train_loader = DataLoader(
        train_ds, batch_size=cfg.BATCH_SIZE, sampler=sampler,
        num_workers=4, pin_memory=True, drop_last=True
    )
    val_loader = DataLoader(
        val_ds, batch_size=cfg.BATCH_SIZE, shuffle=False,
        num_workers=4, pin_memory=True
    )
    test_loader = DataLoader(
        test_ds, batch_size=cfg.BATCH_SIZE, shuffle=False,
        num_workers=4, pin_memory=True
    )

    # Model, criterion, optimizer, scheduler
    model = create_model(cfg.MODEL_TYPE)

    # Use combined loss for better stability
    criterion = CombinedLoss(focal_weight=0.7, ce_weight=0.3)

    optimizer = optim.AdamW(
        model.parameters(),
        lr=cfg.LEARNING_RATE,
        weight_decay=cfg.WEIGHT_DECAY,
        betas=(0.9, 0.999)
    )

    # Scheduler with warmup
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=cfg.LEARNING_RATE * 10,
        epochs=cfg.NUM_EPOCHS,
        steps_per_epoch=len(train_loader),
        pct_start=0.1,  # 10% warmup
        anneal_strategy='cos'
    )

    # Training loop with detailed monitoring
    best_bal = -float("inf")
    patience = 0
    best_metrics = {}

    print("\n" + "="*60)
    print("STARTING TRAINING WITH IMPROVED MONITORING")
    print("="*60)

    for epoch in range(cfg.NUM_EPOCHS):
        # Unfreeze all layers after warmup
        if epoch == cfg.WARMUP_EPOCHS:
            for param in model.parameters():
                param.requires_grad = True
            print(f"✓ Unfroze all layers at epoch {epoch+1}")

        # Training
        tr_loss, tr_acc, tr_bal = train_epoch(model, train_loader, criterion, optimizer, epoch)

        # Validation
        val_results = evaluate(model, val_loader, criterion)

        # Step scheduler
        if isinstance(scheduler, optim.lr_scheduler.OneCycleLR):
            # OneCycleLR steps per batch
            pass
        else:
            scheduler.step()

        # Print detailed metrics
        print(f"\nEpoch {epoch+1}/{cfg.NUM_EPOCHS}:")
        print(f"  Train - Loss: {tr_loss:.4f}, Acc: {tr_acc:.3f}, Bal: {tr_bal:.3f}")
        print(f"  Val   - Loss: {val_results['loss']:.4f}, Acc: {val_results['accuracy']:.3f}, Bal: {val_results['balanced_accuracy']:.3f}")
        print(f"  Val   - Precision: {val_results['precision']:.3f}, Recall: {val_results['recall']:.3f}, F1: {val_results['f1']:.3f}")
        print(f"  Threshold: {val_results['threshold']:.3f}, LR: {optimizer.param_groups[0]['lr']:.6f}")

        # Check for improvement
        current_bal = val_results['balanced_accuracy']
        if current_bal > best_bal + cfg.MIN_DELTA:
            best_bal = current_bal
            best_metrics = val_results.copy()
            patience = 0

            # Save checkpoint
            save_checkpoint(model, optimizer, scheduler, epoch, val_results, 'best.pth')
            print(f"  ✅ New best balanced accuracy: {current_bal:.3f}")
        else:
            patience += 1
            print(f"  ⏳ No improvement. Patience: {patience}/{cfg.PATIENCE}")

            if patience >= cfg.PATIENCE:
                print(f"  🛑 Early stopping at epoch {epoch+1}")
                break

    # Load best model
    print("\n" + "="*60)
    print("LOADING BEST MODEL FOR FINAL EVALUATION")
    print("="*60)

    ckpt = load_checkpoint('best.pth', map_location=device)
    if ckpt is not None and 'model_state_dict' in ckpt:
        model.load_state_dict(ckpt['model_state_dict'])
        print(f"✓ Loaded best model from epoch {ckpt['epoch']+1}")
        print(f"✓ Best validation balanced accuracy: {ckpt['metrics']['balanced_accuracy']:.3f}")
    else:
        print("⚠ Warning: Using current model (no checkpoint loaded)")

    # Final test evaluation
    print("\n" + "="*60)
    print("FINAL TEST EVALUATION")
    print("="*60)

    test_results = evaluate(model, test_loader, criterion, return_probs=True)

    # Print comprehensive results
    print(f"\n🎯 FINAL TEST RESULTS:")
    print(f"  Accuracy: {test_results['accuracy']:.3f} ({test_results['accuracy']*100:.1f}%)")
    print(f"  Balanced Accuracy: {test_results['balanced_accuracy']:.3f} ({test_results['balanced_accuracy']*100:.1f}%)")
    print(f"  Precision: {test_results['precision']:.3f}")
    print(f"  Recall: {test_results['recall']:.3f}")
    print(f"  F1-Score: {test_results['f1']:.3f}")
    print(f"  Optimal Threshold: {test_results['threshold']:.3f}")

    # Detailed confusion matrix analysis
    cm = test_results['confusion_matrix']
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
        npv = tn / (tn + fn) if (tn + fn) > 0 else 0

        print(f"\n📊 MEDICAL METRICS:")
        print(f"  Sensitivity (MCI Detection): {sensitivity:.3f} ({sensitivity*100:.1f}%)")
        print(f"  Specificity (CN Detection): {specificity:.3f} ({specificity*100:.1f}%)")
        print(f"  Positive Predictive Value: {ppv:.3f} ({ppv*100:.1f}%)")
        print(f"  Negative Predictive Value: {npv:.3f} ({npv*100:.1f}%)")

    print(f"\n📈 CONFUSION MATRIX:")
    print("     Predicted")
    print("      CN  MCI")
    print(f" CN  {cm[0,0]:3d} {cm[0,1]:3d}")
    print(f"MCI  {cm[1,0]:3d} {cm[1,1]:3d}")

    # Calculate AUC
    if 'probabilities' in test_results and len(np.unique(test_results['labels'])) > 1:
        try:
            from sklearn.metrics import roc_auc_score
            test_auc = roc_auc_score(test_results['labels'], test_results['probabilities'])
            print(f"\n🎯 AUC-ROC: {test_auc:.3f}")
        except:
            pass

    print(f"\n✅ TRAINING COMPLETED SUCCESSFULLY!")
    print(f"📄 Best model saved as 'best.pth'")

    return test_results

if __name__ == "__main__":
    main()

OASIS Ultimate Pipeline v4.3 - IMPROVED (Training Issues Fixed)
✓ Device: cuda
  GPU: NVIDIA L4
  Memory: 23.80 GB
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Initially found 840 files | CN=754, MCI=86
📊 Original CN samples: 754
🎯 Randomly sampling 440 CN cases...
✅ Balanced dataset: CN=440, MCI=86
📊 Final class imbalance ratio: 5.12:1 (CN:MCI)
✓ Training class distribution: CN=308, MCI=60
✓ Class weights: CN=0.597, MCI=5.520

STARTING TRAINING WITH IMPROVED MONITORING



Epoch 1/60:
  Train - Loss: 0.3798, Acc: 0.523, Bal: 0.517
  Val   - Loss: 0.2424, Acc: 0.810, Bal: 0.670
  Val   - Precision: 0.816, Recall: 0.810, F1: 0.813
  Threshold: 0.491, LR: 0.000012
  ✅ New best balanced accuracy: 0.670



Epoch 2/60:
  Train - Loss: 0.3632, Acc: 0.528, Bal: 0.527
  Val   - Loss: 0.2355, Acc: 0.380, Bal: 0.598
  Val   - Precision: 0.824, Recall: 0.380, F1: 0.408
  Threshold: 0.425, LR: 0.000012
  ⏳ No improvement. Patience: 1/15



Epoch 3/60:
  Train - Loss: 0.3477, Acc: 0.574, Bal: 0.571
  Val   - Loss: 0.2212, Acc: 0.608, Bal: 0.734
  Val   - Precision: 0.860, Recall: 0.608, F1: 0.656
  Threshold: 0.425, LR: 0.000012
  ✅ New best balanced accuracy: 0.734



Epoch 4/60:
  Train - Loss: 0.3698, Acc: 0.483, Bal: 0.475
  Val   - Loss: 0.2163, Acc: 0.620, Bal: 0.742
  Val   - Precision: 0.862, Recall: 0.620, F1: 0.668
  Threshold: 0.411, LR: 0.000012
  ✅ New best balanced accuracy: 0.742


...


Epoch 27/60:
  Train - Loss: 0.3386, Acc: 0.602, Bal: 0.594
  Val   - Loss: 0.1972, Acc: 0.747, Bal: 0.756
  Val   - Precision: 0.848, Recall: 0.747, F1: 0.776
  Threshold: 0.441, LR: 0.000012
  ⏳ No improvement. Patience: 12/15



Epoch 28/60:
  Train - Loss: 0.3419, Acc: 0.577, Bal: 0.565
  Val   - Loss: 0.2010, Acc: 0.633, Bal: 0.780
  Val   - Precision: 0.886, Recall: 0.633, F1: 0.678
  Threshold: 0.353, LR: 0.000012
  ⏳ No improvement. Patience: 13/15



Epoch 29/60:
  Train - Loss: 0.3202, Acc: 0.582, Bal: 0.573
  Val   - Loss: 0.1970, Acc: 0.722, Bal: 0.772
  Val   - Precision: 0.859, Recall: 0.722, F1: 0.756
  Threshold: 0.436, LR: 0.000012
  ⏳ No improvement. Patience: 14/15



Epoch 30/60:
  Train - Loss: 0.3219, Acc: 0.585, Bal: 0.589
  Val   - Loss: 0.1992, Acc: 0.747, Bal: 0.818
  Val   - Precision: 0.882, Recall: 0.747, F1: 0.779
  Threshold: 0.437, LR: 0.000012
  ⏳ No improvement. Patience: 15/15
  🛑 Early stopping at epoch 30

LOADING BEST MODEL FOR FINAL EVALUATION
✓ Loaded best model from epoch 15
✓ Best validation balanced accuracy: 0.863

FINAL TEST EVALUATION



🎯 FINAL TEST RESULTS:
  Accuracy: 0.873 (87.3%)
  Balanced Accuracy: 0.801 (80.1%)
  Precision: 0.882
  Recall: 0.873
  F1-Score: 0.877
  Optimal Threshold: 0.464

📊 MEDICAL METRICS:
  Sensitivity (MCI Detection): 0.692 (69.2%)
  Specificity (CN Detection): 0.909 (90.9%)
  Positive Predictive Value: 0.600 (60.0%)
  Negative Predictive Value: 0.938 (93.8%)

📈 CONFUSION MATRIX:
     Predicted
      CN  MCI
 CN   60   6
MCI    4   9

🎯 AUC-ROC: 0.760

✅ TRAINING COMPLETED SUCCESSFULLY!
📄 Best model saved as 'best.pth'
